In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import seaborn as sns

# Suponha que você tenha um DataFrame com os resultados
# Cada coluna é um dataset e cada linha é um regressor


caminho_arquivo = './resultados.csv'
df = pd.read_csv(caminho_arquivo)
df.set_index('Regressor', inplace=True)

#passar o dataframe para formato float
df = df.astype(float)

df


,ANDRO,ATP1D,ATP7D,WQ,OSALES,SCM1D,SCM20D,RF1,RF2,OES10,OES97
Regressor,,,,,,,,,,,
DT,0.9069,0.7278,0.8404,1.1252,1.0162,0.7120,0.7907,0.3517,0.3333,0.7798,0.8786
HCT DT,0.7613,0.7037,0.8438,1.1223,1.0120,0.6851,0.7786,0.2573,0.3237,0.8272,0.8290
RCT DT,0.7912,0.7180,0.8341,1.1293,1.0144,0.6950,0.7849,0.2809,0.3436,0.7697,0.8204
RF,0.7416,0.6284,0.7526,0.9489,0.8540,0.5772,0.6381,0.3542,0.4435,0.6462,0.7150
HCT RF,0.7450,0.6038,0.7227,0.9509,0.8544,0.5697,0.6419,0.2938,0.3222,0.6787,0.7519
RCT RF,0.7493,0.6112,0.7390,0.9541,0.8630,0.5727,0.6403,0.2638,0.3434,0.6224,0.6942
RC-DT,0.7503,0.7243,0.8699,1.1260,1.0476,0.6998,0.8391,0.2701,0.3476,0.7602,0.8170
HCT RC-DT,0.7683,0.7180,0.8263,1.1181,1.0373,0.6924,0.7957,0.2622,0.3208,0.8404,0.8540
RCT RC-DT,0.8413,0.7130,0.8560,1.1252,1.0131,0.6862,0.7926,0.2807,0.3098,0.7635,0.8093


In [3]:
# Rankear os modelos em cada dataset
ranks = df.apply(lambda x: stats.rankdata(x, method='min'), axis=0)
# print("Rankings:\n", ranks)

# Calcular a média dos rankings para cada modelo
mean_ranks = ranks.mean(axis=1)
print("Mean Rankings:\n", mean_ranks)

# Realizar o teste de Friedman
friedman_stat, p_value = stats.friedmanchisquare(*ranks.values.T)
print(f"Friedman Test: estatística={friedman_stat}, valor p={p_value}")

# Função para calcular o Critical Difference (CD)
def compute_cd(k, N, alpha=0.05):
    from scipy.stats import rankdata, t
    q_alpha = {
        0.01: [0, 0, 0, 0, 2.576, 2.913, 3.113, 3.255, 3.364, 3.452, 3.526, 3.590, 3.646, 3.696, 3.741, 3.781, 3.818, 3.853, 3.884, 3.914],
        0.05: [0, 0, 0, 0, 2.326, 2.569, 2.728, 2.850, 2.949, 3.031, 3.102, 3.164, 3.219, 3.268, 3.313, 3.354, 3.391, 3.426, 3.458, 3.488]
    }
    if alpha not in q_alpha:
        raise ValueError("Alpha value not supported")
    q = q_alpha[alpha][k]
    return q * (k * (k + 1) / (6.0 * N)) ** 0.5

# Se o valor p < 0.05, realizar o teste de Nemenyi e gerar o diagrama CD
if p_value < 0.05:
    nemenyi_result = sp.posthoc_nemenyi_friedman(ranks.T)
    print("Nemenyi Test Results:\n", nemenyi_result)
    
    # Calculando o valor de CD
    cd = compute_cd(len(df), df.shape[1])  # Número de datasets
    print(f"Critical Difference (CD): {cd}")

    # Gerando o gráfico CD manualmente
    def graph_ranks(avranks, names, cd, width=6, textspace=1.5):
        k = len(avranks)
        # Sort averanks and names together
        avranks, names = zip(*sorted(zip(avranks, names)))
        # Margin spaces
        fig, ax = plt.subplots(figsize=(width, k + 1))
        ax.set_xlim(0, width)
        ax.set_ylim(0, k + 1)
        # Plot average ranks
        for i, (rank, name) in enumerate(zip(avranks, names)):
            ax.text(width / 2 - textspace, i + 1, name, horizontalalignment='right', verticalalignment='center')
            ax.text(width / 2 - textspace + 0.1, i + 1, f"{rank:.2f}", horizontalalignment='left', verticalalignment='center')
        # Draw lines for CD
        ax.plot([width / 2 - cd / 2, width / 2 + cd / 2], [k + 0.5, k + 0.5], color='black', lw=1.5)
        ax.text(width / 2, k + 0.8, f"CD = {cd:.2f}", horizontalalignment='center')
        # Hide the axes
        ax.axis('off')
        plt.show()

    # Plotting the graph
    graph_ranks(mean_ranks.values, mean_ranks.index, cd=cd, width=10, textspace=1.5)
else:
    print("Não há diferenças significativas entre os modelos.")

Mean Rankings:
 Regressor
DT           13.545455
HCT DT       10.545455
RCT DT       12.363636
RF            6.000000
HCT RF        4.545455
RCT RF        4.727273
RC-DT        12.727273
HCT RC-DT    11.636364
RCT RC-DT    11.181818
RC-RF         7.545455
HCT RC-RF     6.545455
RCT RC-RF     4.363636
XGB-L         7.727273
XGB-G         7.727273
HCT XGB-G     7.818182
RCT XGB-G     6.727273
dtype: float64
Friedman Test: estatística=0.8323494687131158, valor p=0.9999263212256577
Não há diferenças significativas entre os modelos.
